# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from botbidder import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


In [2]:
models = Models.from_conf(conf.load('./config/TF1.x/default_tf1x.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/TF1.x/default_tf1x.conf'))  # Load sampling strategies

Loading config file d:\GitHub\ben\src\./config/TF1.x/default_tf1x.conf



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/GIB/binfo_V2-1420000
INFO:tensorflow:Restoring parameters from ..\models/lead/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\models/lead/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/play

Loading config file d:\GitHub\ben\src\./config/TF1.x/default_tf1x.conf


### Running through an example

In [3]:

vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=2, dealer=0, ddsolver=dds, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H',
  'insta_score': 0.474,
  'adjustment': 24,
  'expected_score': 206,
  'expected_tricks': 10.5,
  'alert': 'False'},
 {'call': 'PASS',
  'insta_score': 0.515,
  'adjustment': 26,
  'expected_score': 39,
  'expected_tricks': 8.6,
  'alert': 'False'}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.941, 'alert': 'False'}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['x.Q9xx.AKJ9xx.98 QJ98xxx.xx.Q.QTx x.AKJT8x.xxx.Kxx AKTx.x.T8x.AJxxx - 0.71387',
 'A.9xx.AQT9xx.9xx QJ98xxx..Kx.J8xx x.AKJT8x.xxx.Kxx KTxx.Qxxx.J8.AQT - 0.68750',
 '.Q9xx.AKJ98xx.Tx QJ8xxxx.xx.x.QJ9 x.AKJT8x.xxx.Kxx AKT9x.x.QT.A8xxx - 0.66357',
 'J.xxxx.AKQT9xx.x AT9xxxx.Q.x.QJ9x x.AKJT8x.xxx.Kxx KQ8x.9x.J8.AT8xx - 0.65381',
 'x.9xxx.KQJT8x.Ax AQT98xx.x.9.QTxx x.AKJT8x.xxx.Kxx KJxx.Qx.Axx.J98x - 0.64502',
 'Q.Qxxx.K8xxx.A8x AT8xxxx.9x.J.JTx x.AKJT8x.xxx.Kxx KJ9x.x.AQT9.Q9xx - 0.64307',
 'A.Q9x.QJT8x.QT8x KJT98xx.x.Kx.Jxx x.AKJT8x.xxx.Kxx Qxxx.xxx.A9x.A9x - 0.63965',
 'Q.Qxxx.AQJ8x.J9x JT98xxx.x.xx.AQx x.AKJT8x.xxx.Kxx AKxx.9x.KT9.T8xx - 0.63525',
 'K8.xxx.KJ9xx.A8x QJ9xxxx.Q.T.J9xx x.AKJT8x.xxx.Kxx ATx.9xx.AQ8x.QTx - 0.62598',
 'Kx.Qxx.AKT98x.8x QJT98xx..Jx.QJxx x.AKJT8x.xxx.Kxx Axx.9xxx.Qx.AT9x - 0.62549',
 '9.Qxx.AKJ98x.JT8 AQJT8xx.x.QT.9xx x.AKJT8x.xxx.Kxx Kxxx.9xx.xx.AQxx - 0.62256',
 'A9.9xx.AQT98x.8x KQ8xxxx.Qx.x.Q9x x.AKJT8x.xxx.Kxx JTx.xx.KJx.AJTxx - 0.60840',
 'Ax.Qxx.A98xxx.

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: 1.0
x.AKJT8x.xxx.Kxx AKx.Qx.AQ8x.AJ8x Qxxxx.x.T9xx.Txx JT98.9xxx.KJ.Q9x
x.AKJT8x.xxx.Kxx KQT.xx.AKQJx.AQ8 8xxx.x.98x.T9xxx AJ9xx.Q9xx.Tx.Jx
x.AKJT8x.xxx.Kxx AK.xxx.AKQT.ATxx T98xxx.Qx.J8.J9x QJxx.9x.9xxx.Q8x
x.AKJT8x.xxx.Kxx ATx.xx.AKQJT.AQ9 K9xxx.9.xxx.JTxx QJ8x.Qxxx.98.8xx
x.AKJT8x.xxx.Kxx AK.Q9x.AQ9.AQ9xx QTxxxx.xx.Tx.Txx J98x.xx.KJ8xx.J8
x.AKJT8x.xxx.Kxx KT8.Qx.AKQTx.AQx J9xxx.9.8xx.98xx AQxx.xxxx.J9.JTx
x.AKJT8x.xxx.Kxx KQ.Qxx.AKQxx.ATx J9xxxx.x.T8.9xxx AT8x.9xx.J9x.QJ8
x.AKJT8x.xxx.Kxx KQx.Q9x.AK9.AQxx T8xxxx..QTx.J8xx AJ9.xxxx.J8xx.T9
x.AKJT8x.xxx.Kxx AKx.9x.AQJx.AQxx J9xxx.x.Txx.J8xx QT8x.Qxxx.K98.T9
x.AKJT8x.xxx.Kxx QJx.Q9x.AKQx.AQJ T9xxx..T98xx.xxx AK8x.xxxx.J.T98x
x.AKJT8x.xxx.Kxx AK9.Qxx.KJ8.AQJ9 8xxxx.x.Qxxx.xxx QJTx.9xx.AT9.T8x
x.AKJT8x.xxx.Kxx AKx.Q9x.AKxx.Axx J98xx.x.JT98.Txx QTxx.xxx.Qx.QJ98
x.AKJT8x.xxx.Kxx AQx.Qx.AK8x.AQxx T8xxx.xx.Txx.J9x KJ9x.9xx.QJ9.T8x
x.AKJT8x.xxx.Kxx AK.Qxx.A9xx.AQJx JTxxxx.x.8x.T8xx Q98x.9xx.KQJT.9x
x.AKJT8x.xxx.Kxx KQJ.Qxx.AKJ.A

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [7]:
hands = random_deal().split()
hands # N, E, S, W

['KQJ.T72.AJ9.AK52',
 'A76542.A84.Q72.6',
 '.KQJ63.KT8653.93',
 'T983.95.4.QJT874']

In [8]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0, dds, False) for i, hand in enumerate(hands)]

In [9]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['1C', '1S', '2H', '3S', '4H', '4S', '5H', 'PASS', 'PASS', 'PASS']